# Analysing New York City Taxi Data with Spark

In [3]:
 pip install shapely

Note: you may need to restart the kernel to use updated packages.


In [101]:
from collections import defaultdict

from pyspark.sql.window import Window
from pyspark.sql.functions import (
    lead, lag, unix_timestamp, 
    when, col, sum
)
import pyspark
import json
from shapely import *
from shapely.geometry import shape


builder = pyspark.sql.SparkSession.builder.appName("DF2_Practice") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Loading data

In [102]:
# Load GeoJSON file
with open('nyc-boroughs.geojson', 'r') as f:
    data = json.load(f)

boroughs = defaultdict(list)
for feature in data['features']:
    borough = feature['properties']['borough']
    geom = shape(feature['geometry'])

    boroughs[borough].append(geom)

for borough, geoms in boroughs.items():
    boroughs[borough] = unary_union(geoms)

In [103]:
iso_df = (spark.read
          .option("header","true")
          .option("inferSchema","true")
          .csv("sample.csv"))

## Preprocessing

In [104]:
window = Window.orderBy("pickup_datetime").partitionBy("medallion")

iso_df = iso_df.withColumn(
    "prev_pickup", lag("dropoff_datetime").over(window)
).withColumn(
    "delay", unix_timestamp("pickup_datetime") - unix_timestamp("prev_pickup")
).fillna(
    0, subset="delay"
).withColumn(
    "delay", when(col("delay") >= 14400, 0).otherwise(col("delay"))
)

In [105]:
def lkp_dict(long, lat):
    point = Point(long, lat)
    for borough, geom in boroughs.items():
        if geom.contains(point):
            return borough
    return None

lkp_udf = udf(lkp_dict, StringType())

iso_df = iso_df.withColumn(
    "pickup_bur", 
    lkp_udf(iso_df["pickup_longitude"], iso_df["pickup_latitude"])
).withColumn(
    "dropoff_bur", 
    lkp_udf(iso_df["dropoff_longitude"], iso_df["dropoff_latitude"])
)

## Query 1

In [106]:
has_passengers = when(col("passenger_count") > 0, 1).otherwise(0) 
no_passengers = when(col("passenger_count") == 0, 1).otherwise(0) 

iso_df.groupBy("medallion").agg(
    sum("delay").alias("idle_time"),
    sum(col("trip_time_in_secs") * has_passengers).alias("passenger_road_time"),
    sum(col("trip_time_in_secs") * no_passengers).alias("empty_road_time")
).withColumn(
    "utilization", 
    col("passenger_road_time") / (
        col("idle_time") + col("empty_road_time") + col("passenger_road_time")
    )
).select("medallion", "utilization").show()

+--------------------+-------------------+
|           medallion|        utilization|
+--------------------+-------------------+
|002E3B405B6ABEA23...| 0.3886363636363636|
|0030AD2648D81EE87...| 0.7333333333333333|
|0036961468659D0BF...|0.37213740458015265|
|0038EF45118925A51...|0.41935483870967744|
|0053334C798EC6C8E...| 0.2608695652173913|
|005DED7D6E6C45441...| 0.4935400516795866|
|005F00B38F46E2100...| 0.3060217176702863|
|009E68CADCB1BCF73...|                1.0|
|00BD5D1AD3A96C997...|0.44854881266490765|
|00FB3D49C3DE5E002...| 0.5070993914807302|
|012B65864B3BE97D6...|  0.511520737327189|
|012F172C0351A4767...| 0.2848101265822785|
|01389E9CF7758ECAC...|0.34532374100719426|
|019AFB33C3153481B...|0.43252595155709345|
|01BD10395EF30144C...|0.42105263157894735|
|01C905F5CF4CD4D36...| 0.5304054054054054|
|01D13A056D9A26F84...| 0.3333333333333333|
|01D8C877762B42B4F...| 0.5126182965299685|
|01DDF05F9C01F581E...| 0.3877995642701525|
|01F9F5C3B207C3D05...|0.31802721088435376|
+----------

## Query 2

In [111]:
iso_df.filter(
    (iso_df.delay > 0) & col("dropoff_bur").isNotNull()
).groupBy("dropoff_bur").agg(
    avg("delay").alias("borough_delay")
).orderBy("borough_delay").show()

+-------------+------------------+
|  dropoff_bur|     borough_delay|
+-------------+------------------+
|Staten Island|             780.0|
|    Manhattan| 1127.095106255612|
|     Brooklyn|1920.1200297287253|
|       Queens|2041.3681172291297|
|        Bronx| 2192.818791946309|
+-------------+------------------+



## Query 3

In [108]:
qry3 = iso_df.filter(iso_df.dropoff_bur == iso_df.pickup_bur).count()

print(f"Amount of trips that start and end in the same borough: {qry3}")

Amount of trips that start and end in the same borough: 86074


## Query 4

In [109]:
qry4 = iso_df.filter(iso_df.dropoff_bur != iso_df.pickup_bur).count()

print(f"Amount of trips that do not start and end in the same borough: {qry4}")

Amount of trips that do not start and end in the same borough: 11433
